## This notebook read the "constructors" table from the bronze layer and transform it to the silver layer

In [ ]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "delta")
bronze_catalog    = "f1_bronze"
silver_catalog    = "f1_silver"
bronze_schema     = "bronze"
silver_schema     = "silver"
bronze_table_dlt = "f1_constructors_dlt"
bronze_table_par = "f1_constructors_par"
silver_table_dlt = "f1_constructors_dlt"
silver_table_par = "f1_constructors_par"

# ----------
##### Step 1 - Read the bronze table using the spark dataframe reader
#  ----------

In [ ]:
constructor_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")

# ----------
##### Step 2 - Drop unwanted columns from the dataframe
#  ----------

In [ ]:
from pyspark.sql.functions import col

In [ ]:
constructor_dropped_df = constructor_df.drop(col('url'))

# ----------
##### Step 3 - Rename columns and add ingestion date
#  ----------

In [ ]:

from pyspark.sql.functions import current_timestamp

In [ ]:
constructor_final_df = constructor_dropped_df.withColumnRenamed("constructorId", "team_id") \
                                             .withColumnRenamed("constructorRef", "team_ref") \
                                             .withColumn("ingestion_date", current_timestamp())

# ----------
##### Step 4 Write output in selected format
#  ----------

In [ ]:
if target_type == 'file':
    if target_format == 'parquet':
        constructor_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/constructors")
elif target_type == 'table':
    if target_format == 'parquet':
        constructor_final_df.write.mode("overwrite").format("parquet").saveAsTable(f"{silver_catalog}.{silver_schema}.{silver_table_par}")
    elif  target_format == 'delta':
        constructor_final_df.write.mode("overwrite").format("delta").saveAsTable(f"{silver_catalog}.{silver_schema}.{silver_table_dlt}")

In [ ]:
if target_type == 'file':
    if target_format == 'parquet':
        constructor_read_df = spark.read.parquet(f"{silver_folder_path}/constructors")
elif target_type == 'table':
    if target_format == 'parquet':
        constructor_read_df = spark.read.table(f"{silver_catalog}.{silver_schema}.{silver_table_par}")
    elif  target_format == 'delta':
        constructor_read_df = spark.read.table(f"{silver_catalog}.{silver_schema}.{silver_table_dlt}")